# Multi-label classification -- top-push loss

In [ ]:
%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import os, sys, time
import pickle as pkl
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from scipy.optimize import check_grad
from scipy.misc import logsumexp
from scipy.special import expit as sigmoid

from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer, label_ranking_loss

import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

In [ ]:
sys.path.append('src')
from evaluate import avgPrecisionK, evaluatePrecision, evaluateF1, evaluateRankingLoss, f1_score_nowarn
from datasets import create_dataset, dataset_names, nLabels_dict

In [ ]:
dataset_names

In [ ]:
data_ix = 1

In [ ]:
dataset_name = dataset_names[data_ix]
nLabels = nLabels_dict[dataset_name]
print(dataset_name, nLabels)

In [ ]:
data_dir = 'data'
SEED = 918273645
fmodel_base = os.path.join(data_dir, 'tp-' + dataset_name + '-base.pkl')
fmodel_prec = os.path.join(data_dir, 'tp-' + dataset_name + '-prec.pkl')
fmodel_noclsw = os.path.join(data_dir, 'tp-' + dataset_name + '-noclsw.pkl')
fperf_base = os.path.join(data_dir, 'perf-tp-base.pkl')
fperf_prec = os.path.join(data_dir, 'perf-tp-prec.pkl')
fperf_noclsw = os.path.join(data_dir, 'perf-tp-noclsw.pkl')

Load dataset.

In [ ]:
X_train, Y_train = create_dataset(dataset_name, train_data=True, shuffle=True, random_state=SEED)
X_test,  Y_test  = create_dataset(dataset_name, train_data=False)

In [ ]:
print('%-45s %s' % ('Dataset:', dataset_name))
print('%-45s %d' % ('Number of training examples:', X_train.shape[0]))
print('%-45s %d' % ('Number of test examples:', X_test.shape[0]))
print('%-45s %d' % ('Number of features:', X_train.shape[1]))
print('%-45s %d' % ('Number of labels:', Y_train.shape[1]))
print('%-45s %d' % ('Average number of positive labels (train):', np.mean(np.sum(Y_train, axis=1))))
print('%-45s %d' % ('Average number of positive labels (test):', np.mean(np.sum(Y_test, axis=1))))

## top-push loss

Multi-label learning with top push loss.

In [ ]:
#n = 100000
#m = 8000
#randM = np.random.rand(n, m)

In [ ]:
#%%timeit
#print(np.sum(randM))

In [ ]:
#%%timeit
#print(np.dot(np.ones(n), np.dot(randM, np.ones(m))))  # more efficient than np.sum()

In [ ]:
def obj_toppush(w, X, Y, C, r=1, weighting=True):
    """
        Objective with L2 regularisation and top push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, C = 1 / lambda
            - r: parameter for log-sum-exp approximation
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == K * D)
    assert(r > 0)
    assert(C > 0)
    
    W = w.reshape(K, D)  # theta
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    
    # instead of using diagonal matrix to scale each row of a matrix with a different factor,
    # we use Mat * Vec[:, None] which is more memory efficient
    
    if weighting is True:
        KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    else:
        KPosAll = np.ones(N)
        
    A_diag = 1.0 / KPosAll
    AY = Y * A_diag[:, None]
    
    T1 = np.dot(X, W.T)  # N by K
    #m0 = np.max(T1)  # underflow in np.exp(r*T1 - m1)
    m0 = 0.5 * (np.max(T1) + np.min(T1))
    m1 = r * m0
    #print('----------------')
    #print(m0, np.min(T1))
    
    T2 = np.multiply(1 - Y, np.exp(r * T1 - m1))  # N by K
    B_tilde_diag = np.dot(T2, np.ones(K))
    #print(np.max(B_tilde_diag), np.min(B_tilde_diag))  # big numbers here, can cause overflow in T3
    
    #T3 = np.exp(-T1 + m0) * np.power(B_tilde_diag, 1.0 / r)[:, None]
    #T4 = np.multiply(AY, np.log1p(T3))
    T3 = (-T1 + m0) + (1.0 / r) * np.log(B_tilde_diag)[:, None]
    #print(np.min(T3), np.max(T3))
    m2 = 0.5 * (np.min(T3) + np.max(T3))
    #T4 = np.logaddexp(0, T3)
    T4 = np.logaddexp(-m2, T3-m2) + m2
    T5 = np.multiply(AY, T4)  
    
    J = np.dot(w, w) * 0.5 / C + np.dot(np.ones(N), np.dot(T5, np.ones(K))) / N
    
    #T5 = 1.0 / (1.0 + np.divide(1.0, T3))
    #T5 = np.divide(T3, 1 + T3)
    T6 = np.exp(T3 - T4)
    O_diag = np.dot(np.multiply(Y, T6), np.ones(K))
    T7 = A_diag * (1.0 / B_tilde_diag) * O_diag
    
    G1 = np.dot(np.multiply(AY, T6).T, -X)
    G2 = np.dot((T2 * T7[:, None]).T, X)
    
    G = W / C + (G1 + G2) / N
    
    return (J, G.ravel())

In [ ]:
def obj_toppush_loop(w, X, Y, C, r=1, weighting=True):
    """
        Objective with L2 regularisation and top push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, C = 1 / lambda
            - r: parameter for log-sum-exp approximation
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == K * D)
    assert(r > 0)
    assert(C > 0)
    
    W = w.reshape(K, D)  # theta
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    if weighting is True:
        KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    else:
        KPosAll = np.ones(N)
    
    for n in range(N):
        for k in range(K):
            if Y[n, k] == 1:
                s1 = np.sum([np.exp(r * np.dot(W[j, :] - W[k, :], X[n, :])) for j in range(K) if Y[n, j] == 0])
                J += np.log1p(np.power(s1, 1.0 / r)) / KPosAll[n]
    J = np.dot(w, w) * 0.5 / C + J / N
    
    for k in range(K):
        for n in range(N):
            if Y[n, k] == 1:
                t1 = np.sum([np.exp(r * np.dot(W[j, :] - W[k, :], X[n, :])) for j in range(K) if Y[n, j] == 0])
                t2 = -1.0 / (1 + np.power(t1, -1.0 / r))
                G[k, :] = G[k, :] + X[n, :] * t2 / KPosAll[n]
            else:
                sk = 0.0
                for k1 in range(K):
                    if Y[n, k1] == 1:
                        t3 = np.sum([np.exp(r * np.dot(W[j,:] - W[k1, :], X[n, :])) \
                                     for j in range(K) if Y[n, j] == 0])
                        t4 = np.exp(r * np.dot(W[k, :] - W[k1, :], X[n, :]))
                        sk += t4 / (np.power(t3, 1.0 - 1.0 / r) + t3)
                G[k, :] = G[k, :] + X[n, :] * sk / KPosAll[n]
                        
    G = W / C + G / N
    
    return (J, G.ravel())

Check gradient

In [ ]:
#aa = np.array([0,1,2, 0])
#print(aa)
#print([aa[i] for i in range(4) if aa[i] != 0])
#print([aa[i] if aa[i] != 0 else 10 for i in range(4)])

In [ ]:
#X_train = X_train[:50, :]
#Y_train = Y_train[:50, :]
#C = 1
##w0 = np.random.rand(Y_train.shape[1] * X_train.shape[1]) - 0.5
#w0 = 0.001 * np.random.randn(Y_train.shape[1] * X_train.shape[1])

In [ ]:
check_grad(lambda w: obj_toppush(w, X_train, Y_train, C, r=2)[0], 
           lambda w: obj_toppush(w, X_train, Y_train, C, r=2)[1], w0)

In [ ]:
#check_grad(lambda w: obj_toppush_loop(w, X_train, Y_train, C, r=2)[0], 
#           lambda w: obj_toppush_loop(w, X_train, Y_train, C, r=2)[1], w0)

In [ ]:
%%script false
eps = 1.49e-08
w = np.zeros_like(w0)
for i in range(len(w0)):
    wi1 = w0.copy()
    wi2 = w0.copy()
    wi1[i] = wi1[i] - eps
    wi2[i] = wi2[i] + eps
    J1, _ = obj_toppush_loop(wi1, X_train, Y_train, C)
    J2, _ = obj_toppush_loop(wi2, X_train, Y_train, C)
    w[i] = (J2 - J1) / (2 * eps)
    #print(w[i])
J, w1 = obj_toppush_loop(w0, X_train, Y_train, C)
diff = w1 - w
np.sqrt(np.dot(diff, diff))

In [ ]:
#%%script false
print('%15s %15s %15s %15s' % ('J_Diff', 'J_loop', 'J_vec', 'G_Diff'))
for e in range(-6, 10):
    C = 10**(e)
    #w0 = init_var(X_train, Y_train)
    J,  G  = obj_toppush_loop(w0, X_train, Y_train, C, r=2)
    J1, G1 = obj_toppush(w0, X_train, Y_train, C, r=2)
    Gdiff = G1 - G
    #print('%-15g %-15g %-15g' % (J1 - J, J, J1))
    print('%15g %15g %15g %15g' % (J1 - J, J, J1, np.dot(Gdiff, Gdiff)))

In [ ]:
# ValueError: scoring must return a number, NOT an array
def search_TH0(Y_true, Y_pred):
    """
        For a given label, search the best threshold in 100 candidates that sampled uniformly at random
        from all N+1 possible thresholds 
    """
    #SEED = 987654321
    #np.random.seed(SEED)
    N, K = Y_true.shape
    THs = np.zeros(K)
    for k in range(K):
        # compute threshold as the mean of two successive scores
        y = sorted(Y_pred[:, k])
        y1 = np.zeros(N+1)
        y2 = np.zeros(N+1)
        y1[:N] = y
        y1[N] = y[-1] + 1
        y2[1:] = y
        y2[0] = y[0] - 1
        TH = 0.5 * (y1 + y2)
        np.random.shuffle(TH)
        bestTH = 0
        bestF1 = 0
        for th in TH[:100]:
            F1 = f1_score_nowarn(Y_true[:, k], Y_pred[:, k] >= th)
            if F1 > bestF1:
                bestF1 = F1
                bestTH = th
        THs[k] = bestTH
    return THs

In [ ]:
def get_TH(Y_true, Y_pred):
    allPreds = sigmoid(Y_pred)
    ranges = np.arange(0.01, 1, 0.01)
    #F1 = [f1_score_nowarn(Y_true, allPreds >= th, average='samples') for th in ranges]
    F1 = Parallel(n_jobs=4)(delayed(f1_score_nowarn)(Y_true, allPreds >= th, average='samples') for th in ranges)
    bestix = np.argmax(F1)
    return ranges[bestix]

In [ ]:
class MLC_toppush(BaseEstimator):
    """All methods are necessary for a scikit-learn estimator"""
    
    def __init__(self, C=1, r=1, weighting=True):
        """Initialisation"""
        
        assert C > 0
        assert r > 0
        assert type(weighting) == bool
        self.C = C
        self.r = r
        self.weighting = weighting
        self.trained = False
        
    def fit(self, X_train, Y_train):
        """Model fitting by optimising the objective"""
        opt_method = 'L-BFGS-B' #'BFGS' #'Newton-CG'
        options = {'disp': 1, 'maxiter': 10**5, 'maxfun': 10**5} # , 'iprint': 99}
        print('\nC: %g, r: %g' % (self.C, self.r))
            
        N, D = X_train.shape
        K = Y_train.shape[1]
        #w0 = np.random.rand(K * D) - 0.5  # initial guess in range [-1, 1]
        w0 = 0.001 * np.random.randn(K * D)
        opt = minimize(obj_toppush, w0, args=(X_train, Y_train, self.C, self.r, self.weighting), \
                       method=opt_method, jac=True, options=options)
        if opt.success is True:
            self.W = np.reshape(opt.x, (K, D))
            self.trained = True
        else:
            sys.stderr.write('Optimisation failed')
            print(opt.items())
            self.trained = False
            
            
    def decision_function(self, X_test):
        """Make predictions (score is real number)"""
        
        assert self.trained is True, "Can't make prediction before training"
        D = X_test.shape[1]
        return np.dot(X_test, self.W.T)
        
    
    def predict(self, X_test):
        return self.decision_function(X_test)
    #    """Make predictions (score is boolean)"""   
    #    preds = sigmoid(self.decision_function(X_test))
    #    #return (preds >= 0)
    #    assert self.TH is not None
    #    return preds >= self.TH        
        
    # inherit from BaseEstimator instead of re-implement
    #
    #def get_params(self, deep = True):
    #def set_params(self, **params):

In [ ]:
def dump_results(predictor, X_train, Y_train, X_test, Y_test, fname):
    """
        Compute and save performance results
    """
    preds_train = predictor.decision_function(X_train)
    preds_test  = predictor.decision_function(X_test)
    
    print('Training set:')
    perf_dict_train = evaluatePrecision(Y_train, preds_train)
    print()
    print('Test set:')
    perf_dict_test = evaluatePrecision(Y_test, preds_test)
    
    print()
    print('Training set:')
    perf_dict_train.update(evaluateRankingLoss(Y_train, preds_train))
    print(label_ranking_loss(Y_train, preds_train))
    print()
    print('Test set:')
    perf_dict_test.update(evaluateRankingLoss(Y_test, preds_test))
    print(label_ranking_loss(Y_test, preds_test))
    
    # compute F1 score w.r.t. different thresholds
    #TH1 = predictor.cv_results_['mean_test_TH'][clf.best_index_]
    TH2 = np.mean(Y_train, axis=0)
    TH3 = np.mean(TH2)
    
    preds_train_bin = sigmoid(preds_train)
    preds_test_bin  = sigmoid(preds_test)
    
    #F1_train1 = f1_score_nowarn(Y_train, sigmoid(preds_train) >= TH1, average='samples')
    #F1_test1  = f1_score_nowarn(Y_test, sigmoid(preds_test) >= TH1, average='samples')
    #print('\nTrain: %.4f, %f' % (F1_train1, f1_score(Y_train, sigmoid(preds_train) >= TH1, average='samples')))
    #print('\nTest : %.4f, %f' % (F1_test1, f1_score(Y_test, sigmoid(preds_test) >= TH1, average='samples')))
    
    F1_train2 = f1_score_nowarn(Y_train, (preds_train_bin - TH2) >= 0, average='samples')
    F1_test2  = f1_score_nowarn(Y_test, (preds_test_bin - TH2) >= 0, average='samples')
    print('\nTrain: %.4f, %f' % (F1_train2, f1_score(Y_train, (preds_train_bin - TH2) >= 0, average='samples')))
    print('\nTest : %.4f, %f' % (F1_test2, f1_score(Y_test, (preds_test_bin - TH2) >= 0, average='samples')))
    
    F1_train3 = f1_score_nowarn(Y_train, preds_train_bin >= TH3, average='samples')
    F1_test3  = f1_score_nowarn(Y_test, preds_test_bin >= TH3, average='samples')
    print('\nTrain: %.4f, %f' % (F1_train3, f1_score(Y_train, preds_train_bin >= TH3, average='samples')))
    print('\nTest : %.4f, %f' % (F1_test3, f1_score(Y_test, preds_test_bin >= TH3, average='samples')))
    
    #perf_dict_train.update({'F1': [(F1_train1,), (F1_train2,), (F1_train3,)]})
    #perf_dict_test.update( {'F1': [(F1_test1,),  (F1_test2,),  (F1_test3,)]})
    perf_dict_train.update({'F1': [(F1_train2,), (F1_train3,)]})
    perf_dict_test.update( {'F1': [(F1_test2,),  (F1_test3,)]})
    
    perf_dict = {'Train': perf_dict_train, 'Test': perf_dict_test}
    if os.path.exists(fname):
        _dict = pkl.load(open(fname, 'rb'))
        if dataset_name not in _dict:
            _dict[dataset_name] = perf_dict
    else:
        _dict = {dataset_name: perf_dict}
    pkl.dump(_dict, open(fname, 'wb'))
    
    print()
    print(pkl.load(open(fname, 'rb')))

In [ ]:
old_settings = np.seterr(all='ignore')  # seterr to known value
np.seterr(all='raise')
#np.seterr(all='ignore')
#np.seterr(**old_settings)  # restore settings

In [ ]:
#%memit model.fit(X_train[:30], Y_train[:30])
#%mprun -f minimize model.fit(X_train[:100], Y_train[:100])
#%mprun -f _minimize_slsqp model.fit(X_train[:10], Y_train[:10])

Default model.

In [ ]:
if os.path.exists(fmodel_base):
    clf = pkl.load(open(fmodel_base, 'rb'))
else:
    clf = clf = MLC_toppush()
    clf.fit(X_train, Y_train)
    pkl.dump(clf, open(fmodel_base, 'wb'))

In [ ]:
dump_results(clf, X_train, Y_train, X_test, Y_test, fperf_base)

Cross validation w.r.t. average precision@K.

In [ ]:
ranges = range(-6, 7)
parameters = [{'C': sorted([10**(e) for e in ranges] + [3 * 10**(e) for e in ranges])}]
scorer = {'Prec': make_scorer(avgPrecisionK)}#, 'TH': make_scorer(get_TH)}

In [ ]:
if not os.path.exists(fmodel_prec):
    clf = GridSearchCV(MLC_toppush(), parameters, scoring=scorer, cv=10, n_jobs=1, refit='Prec')
    clf.fit(X_train, Y_train)
    pkl.dump(clf, open(fmodel_prec, 'wb'))
else:
    clf = pkl.load(open(fmodel_prec, 'rb'))

In [ ]:
dump_results(clf, X_train, Y_train, X_test, Y_test, fperf_prec)

Cross validation w.r.t. average precision@K, without weighting positive labels in objective.

In [ ]:
if not os.path.exists(fmodel_noclsw):
    clf = GridSearchCV(MLC_toppush(weighting=False), parameters, scoring=scorer, cv=10, n_jobs=1, refit='Prec')
    clf.fit(X_train, Y_train)
    pkl.dump(clf, open(fmodel_noclsw, 'wb'))
else:
    clf = pkl.load(open(fmodel_noclsw, 'rb'))

In [ ]:
dump_results(clf, X_train, Y_train, X_test, Y_test, fperf_noclsw)

Compute threshold for F1:
- approach I: grid search in range(0.01, 1) with step 0.01 during cross-validation.
- approach II: for each label, let the threshold be the average probability (to be +1) in training set.
- approach III: let the threshold be the average of all thresholds computed in approach II.

In [ ]:
%%script false

#TH1 = clf.cv_results_['mean_test_TH'][clf.best_index_]
#print(TH1)

TH2 = np.mean(Y_train, axis=0)
TH3 = np.mean(TH2)

#F1_train1 = f1_score_nowarn(Y_train, sigmoid(preds_train) >= TH1, average='samples')
#F1_test1  = f1_score_nowarn(Y_test, sigmoid(preds_test) >= TH1, average='samples')
#print('Train: %.4f \n' % F1_train1, f1_score(Y_train, sigmoid(preds_train) >= TH1, average='samples'))
#print('Test : %.4f \n' % F1_test1, f1_score(Y_test, sigmoid(preds_test) >= TH1, average='samples'))

F1_train2 = f1_score_nowarn(Y_train, (preds_train - TH2) >= 0, average='samples')
F1_test2  = f1_score_nowarn(Y_test, (preds_test - TH2) >= 0, average='samples')
print('Train: %.4f \n' % F1_train2, f1_score(Y_train, (preds_train - TH2) >= 0, average='samples'))
print('Test : %.4f \n' % F1_test2, f1_score(Y_test, (preds_test - TH2) >= 0, average='samples'))

F1_train3 = f1_score_nowarn(Y_train, preds_train >= TH3, average='samples')
F1_test3  = f1_score_nowarn(Y_test, preds_test >= TH3, average='samples')
print('Train: %.4f \n' % F1_train3, f1_score(Y_train, preds_train >= TH3, average='samples'))
print('Test : %.4f \n' % F1_test3, f1_score(Y_test, preds_test >= TH3, average='samples'))

## Result analysis

Plot the histogram of the $K$-th highest prediction scores, where $K$ is the number of positive labels for a given example.

In [ ]:
%%script false

K_train = Y_train.sum(axis=1).astype(np.int)
K_test = Y_test.sum(axis=1).astype(np.int)

K_pred_train = [sorted(preds_train[n, :])[::-1][K_train[n]-1] for n in range(X_train.shape[0])]
K_pred_test = [sorted(preds_test[n, :])[::-1][K_test[n]-1] for n in range(X_test.shape[0])]

fig = plt.figure(figsize=[10, 3])
plot_data = [K_pred_train, K_pred_test]
plt.suptitle('Histogram on %s dataset' % dataset_name)
for i in range(2):
    ax = plt.subplot(1,2,i+1)
    ax.hist(plot_data[i], bins=20)
    ax.set_title('%s set' % ('Training' if i == 0 else 'Test'))
    ax.set_xlabel('K-th highest prediction score')
    ax.set_ylabel('Frequency')
plt.savefig(dataset_name + '_kscore_hist.svg')

#print(K_pred_train[0], K_train[0])
#print(K_pred_test[0], K_test[0])

#preds_train[0, :]
#preds_test[0, :]

precisions_train = [avgPrecision(Y_train, preds_train, k) for k in range(1, nLabels+1)]
precisions_test  = [avgPrecision(Y_test,  preds_test,  k) for k in range(1, nLabels+1)]

precisionK_train = avgPrecisionK(Y_train, preds_train)
precisionK_test  = avgPrecisionK(Y_test,  preds_test)

plt.figure(figsize=[10,5])
plt.plot(precisions_train, ls='--', c='r', label='Train')
plt.plot(precisions_test,  ls='-',  c='g', label='Test')
plt.plot([precisionK_train for k in range(nLabels)], ls='-', c='r', label='Train, Precision@K')
plt.plot([precisionK_test  for k in range(nLabels)], ls='-', c='g', label='Test, Precision@K')
plt.xticks(np.arange(nLabels), np.arange(1,nLabels+1))
plt.xlabel('k')
plt.ylabel('Precision@k')
plt.legend(loc='best')
plt.title('MLC w. Top-push Loss on ' + dataset_name + ' dataset')
#plt.savefig(dataset_name + '_tp.svg')